<a href="https://colab.research.google.com/github/LoPA607/SOC-2024/blob/main/WEEK%202/resnet18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi
import torch
import torch.nn as nn

Sat Jun 15 17:51:40 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
#ConvBlock
class ConvBlock(nn.Module):
  def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
    super().__init__()
    self.c=nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
    self.bn=nn.BatchNorm2d(out_channels)

  def forward(self, x):
    return self.bn(self.c(x))

#Bottleneck Residual Block
class ResidualBlock(nn.Module):
  def __init__(self, in_channels,out_channels, first=False):
    super().__init__()
    res_channels=out_channels//4
    stride=1

    self.projection=in_channels!=out_channels
    if self.projection:
      self.p=ConvBlock(in_channels, out_channels, 1, 2, 0)
      stride=2
      res_channels=in_channels // 2

    if first:
      self.p=ConvBlock(in_channels, out_channels, 1, 1, 0)
      stride=1
      res_channels=in_channels


    self.c1=ConvBlock(in_channels, res_channels, 1, 1, 0)
    self.c2=ConvBlock(res_channels, res_channels, 3, stride, 1)
    self.c3=ConvBlock(res_channels, out_channels, 1, 1, 0)
    self.relu=nn.ReLU()

  def forward(self, x):
    f=self.relu(self.c1(x))
    f=self.relu(self.c2(f))
    f=self.c3(f)

    if self.projection:
      x=self.p(x)

    h=self.relu(torch.add(f , x))

    return h




In [ ]:
#ResNet
class ResNet(nn.Module):
  def __init__(self, no_blocks, in_channels=3,classes=1000):
    super().__init__()
    out_features=[256, 512, 1024, 2048]
    self.blocks=nn.ModuleList([ResidualBlock(64,256, True)])

    for i in range(len(out_features)):
      if i>0:
         self.blocks.append(ResidualBlock(out_features[i-1], out_features[i]))
      for _ in range(no_blocks[i]-1):
         self.blocks.append(ResidualBlock(out_features[i], out_features[i]))


    self.convl=ConvBlock(in_channels,64 , 72,2,3)
    self.maxpool=nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
    self.avgpool=nn.AdaptiveAvgPool2d((1,1))
    self.fc=nn.Linear(2048, classes)

    self.relu=nn.ReLU()

  def forward(self, x):
    x=self.relu(self.convl(x))
    x=self.maxpool(x)
    for block in self.blocks:
      x=block(x)
    x=self.avgpool(x)
    x=torch.flatten(x,1)
    x=self.fc(x)
    return x




In [ ]:
#no_blocks=[3, 4, 6, 3]
#res=ResNet(no_blocks)
#res(torch.rand(1,3,224,224)).shape

In [ ]:

import torchvision
import torchvision.transforms as transforms



test_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((120, 120)),
    torchvision.transforms.CenterCrop((110, 110)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])




train_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((120, 120)),
    torchvision.transforms.RandomCrop((110, 110)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transforms)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transforms)
valid_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=test_transforms)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=256, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=256, shuffle=False)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=256, shuffle=False)
for images, labels in train_loader:
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    print('Class labels of 10 examples:', labels[:10])
    break




100%|██████████| 170498071/170498071 [00:03<00:00, 48677849.74it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Files already downloaded and verified
Image batch dimensions: torch.Size([256, 3, 110, 110])
Image label dimensions: torch.Size([256])
Class labels of 10 examples: tensor([5, 5, 6, 8, 2, 9, 9, 7, 9, 8])


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = ResNet(no_blocks=[3, 4, 6, 3]).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

model.train()
for epoch in range(20):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print training progress
        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(
                epoch + 1, 20, i + 1, len(train_loader), loss.item()))

# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Test Accuracy: {}%'.format((correct / total) * 100))



Epoch [1/20], Step [100/196], Loss: 1.8396
Epoch [2/20], Step [100/196], Loss: 1.4568
Epoch [3/20], Step [100/196], Loss: 1.4478
Epoch [4/20], Step [100/196], Loss: 1.3708
Epoch [5/20], Step [100/196], Loss: 1.1978
Epoch [6/20], Step [100/196], Loss: 1.1527
Epoch [7/20], Step [100/196], Loss: 1.2049
Epoch [8/20], Step [100/196], Loss: 0.9893
Epoch [9/20], Step [100/196], Loss: 1.3488
Epoch [10/20], Step [100/196], Loss: 0.9615
Epoch [11/20], Step [100/196], Loss: 1.0267
Epoch [12/20], Step [100/196], Loss: 0.9656
Epoch [13/20], Step [100/196], Loss: 1.1986
Epoch [14/20], Step [100/196], Loss: 1.0551
Epoch [15/20], Step [100/196], Loss: 0.8573
Epoch [16/20], Step [100/196], Loss: 0.8657
Epoch [17/20], Step [100/196], Loss: 0.7054
Epoch [18/20], Step [100/196], Loss: 0.8071
Epoch [19/20], Step [100/196], Loss: 1.0044
Epoch [20/20], Step [100/196], Loss: 0.7744
Test Accuracy: 61.39%
